In [1]:
from tensorflow.keras.utils import to_categorical
from PIL import Image
import glob
import matplotlib.pyplot as pyplot
import math
import numpy
import os
import pandas

In [2]:
def get_captcha_label(file_path):
    """
    Precondition: 80x60 dimension CAPTCHA images were generated using the
                  'generator.py' script found in this project folder
    
    Args:
        file_path (str): the path to the CAPTCHA image
    
    Returns:
        the 'label' for each CAPTCHA denoted by the 
        string in the file name before the '_'
        character

        Example: '9876_image.png' -> '9876' 
    """
    try:
        path, file_name = os.path.split(file_path)
        file_name, extension = os.path.splitext(file_name)
        label, _ = file_name.split("_")
        return label
    except Exception as e:
        print('error while parsing %s. %s' % (file_path, e))
        return None, None

In [3]:
def create_captcha_dataframe(captcha_images_directory):
    """
    Args:
        captcha_images_directory (str): the full file path to the folder where the captcha images
                                        were generated
    
    Returns:
        a pandas.DataFrame object storing each captcha file name along with its label
    """
    files = glob.glob(os.path.join(captcha_images_directory, "*.png"))
    attributes = list(map(get_captcha_label, files))

    data_frame = pandas.DataFrame(attributes)
    data_frame['file'] = files
    data_frame.columns = ['label', 'file']
    data_frame = data_frame.dropna()
    
    return data_frame

In [4]:
def shuffle_and_split_data(data_frame):
    """
        Shuffle and split the data into 3 sets: training, validation, and testing.
    
    Args:
        data_frame (pandas.DataFrame): the data to shuffle and split
    
    Returns:
        3 numpy.ndarray objects -> (train_indices, validation_indices, test_indices)
        each hold the index positions for data in the pandas.DataFrame 
    """
    shuffled_indices = numpy.random.permutation(len(data_frame))
    train_up_to = int(len(data_frame) * 0.7)
    train_indices = shuffled_indices[:train_up_to]
    test_indices = shuffled_indices[train_up_to:]

    # Further split up the training data.
    train_up_to = int(train_up_to * 0.7)
    train_indices, validation_indices = train_indices[:train_up_to], train_indices[train_up_to:]
    
    return train_indices, validation_indices, test_indices

In [5]:
def get_captcha_generator(data_frame, indices, for_training, batch_size=16, image_height=100, image_width=100,
                          categories=10):
    """    
    Args:
        data_frame (pandas.DataFrame): contains the file paths to the CAPTCHA images and their labels
        
        indices (int): specifies training indices, testing indices, or validation indices of the DataFrame
        
        for_training (bool): 'True' for training or validation set, 'False' to specify a test set 
        
        batch_size (int): number of data instances to return when iterated upon
        
        image_height (int): height in pixels to resize the CAPTCHA image to
        
        image_width (int): width in pixels to resize the CAPTCHA image to
        
        categories (int): number of possible values for each position in the CAPTCHA image
    
    Returns:
        a generator object for producing CAPTCHA images along with their labels
        
    Yields:
        a pair of lists -> (CAPTCHA images, labels)
    """
    images, labels = [], []
    
    while True:
        for i in indices:
            captcha = data_frame.iloc[i]
            file, label = captcha['file'], captcha['label']
            captcha_image = Image.open(file)
            captcha_image = captcha_image.resize((image_height, image_width))
            captcha_image = numpy.array(captcha_image) / 255.0
            images.append(numpy.array(captcha_image))
            labels.append(numpy.array([numpy.array(to_categorical(int(i), categories)) for i in label]))
            if len(images) >= batch_size:
                yield numpy.array(images), numpy.array(labels)
                images, labels = [], []
        if not for_training:
            break

In [6]:
def display_predictions_from_model(captcha_images, predictions, true_values, total_to_display=30, columns=5):
    """
        Display a plot showing the results of the model's predictions.
        Each subplot will contain the CAPTCHA image, the model's prediction value, and the true value (label).
        
    Args:
        captcha_images (PNG image): the CAPTCHA image file
        
        predictions (EagerTensor): the prediction value made by the model
        
        true_values (EagerTensor): the label associated with the CAPTCHA image
        
        total_to_display (int): total number of subplots
        
        columns (int): number of columns in the plot
    """
    random_indices = numpy.random.permutation(total_to_display)
    rows = math.ceil(total_to_display / columns)

    figure, axes = pyplot.subplots(rows, columns, figsize=(15, 20))
    
    for i, image_index in enumerate(random_indices):
        result = axes.flat[i]
        result.imshow(captcha_images[image_index])
        result.set_title('prediction: {}'.format(
                         ''.join(map(str, predictions[image_index].numpy()))))
        result.set_xlabel('true value: {}'.format(
                          ''.join(map(str, true_values[image_index].numpy()))))
        result.set_xticks([])
        result.set_yticks([])